In [4]:
import numpy as np
import pandas as pd
import folium

In [5]:
# Read CSV
pd.set_option('display.max_columns', None)
raw_data = pd.read_csv("../Resources/Nutrition__Physical_Activity__and_Obesity_-_Behavioral_Risk_Factor_Surveillance_System.csv")
raw_data.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value_Unit,Data_Value_Type,Data_Value,Data_Value_Alt,Data_Value_Footnote_Symbol,Data_Value_Footnote,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,Total,Age(years),Education,Gender,Income,Race/Ethnicity,GeoLocation,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1
0,2012,2012,WY,Wyoming,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,48.5,48.5,NaN,NaN,32.3,64.9,69,NaN,NaN,NaN,NaN,NaN,American Indian/Alaska Native,"(43.235541343, -108.109830353)",OWS,OWS1,Q037,VALUE,56,Race/Ethnicity,American Indian/Alaska Native,RACE,RACENAA
1,2012,2012,DC,District of Columbia,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,31.6,31.6,NaN,NaN,24.0,40.4,243,NaN,NaN,Less than high school,NaN,NaN,NaN,"(38.890371385, -77.031961127)",OWS,OWS1,Q036,VALUE,11,Education,Less than high school,EDU,EDUHS
2,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,35.2,35.2,NaN,NaN,30.7,40.0,598,NaN,25 - 34,NaN,NaN,NaN,NaN,"(32.840571122, -86.631860762)",OWS,OWS1,Q036,VALUE,1,Age (years),25 - 34,AGEYR,AGEYR2534
3,2013,2013,US,National,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,27.9,27.9,NaN,NaN,27.6,28.3,"266,452",NaN,NaN,NaN,Female,NaN,NaN,NaN,PA,PA1,Q047,VALUE,59,Gender,Female,GEN,FEMALE
4,2011,2011,US,National,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,16.9,16.9,NaN,NaN,16.0,17.8,"20,923",NaN,18 - 24,NaN,NaN,NaN,NaN,NaN,PA,PA1,Q047,VALUE,59,Age (years),18 - 24,AGEYR,AGEYR1824


In [6]:
trim_data = raw_data[["YearStart", "LocationAbbr", "LocationDesc", "Question", "Data_Value", "Age(years)", "Education",
                     "Race/Ethnicity", "Gender", "Income", "Total", "StratificationCategory1", "StratificationCategoryId1", "StratificationID1"]]
trim_data.head()

,YearStart,LocationAbbr,LocationDesc,Question,Data_Value,Age(years),Education,Race/Ethnicity,Gender,Income,Total,StratificationCategory1,StratificationCategoryId1,StratificationID1
0,2012,WY,Wyoming,Percent of adults aged 18 years and older who ...,48.5,NaN,NaN,American Indian/Alaska Native,NaN,NaN,NaN,Race/Ethnicity,RACE,RACENAA
1,2012,DC,District of Columbia,Percent of adults aged 18 years and older who ...,31.6,NaN,Less than high school,NaN,NaN,NaN,NaN,Education,EDU,EDUHS
2,2011,AL,Alabama,Percent of adults aged 18 years and older who ...,35.2,25 - 34,NaN,NaN,NaN,NaN,NaN,Age (years),AGEYR,AGEYR2534
3,2013,US,National,Percent of adults who engage in no leisure-tim...,27.9,NaN,NaN,NaN,Female,NaN,NaN,Gender,GEN,FEMALE
4,2011,US,National,Percent of adults who engage in no leisure-tim...,16.9,18 - 24,NaN,NaN,NaN,NaN,NaN,Age (years),AGEYR,AGEYR1824


In [7]:
trim_data = trim_data.loc[(trim_data["Question"] == "Percent of adults aged 18 years and older who have obesity")]

In [8]:
overall_data = trim_data[["YearStart", "LocationAbbr", "LocationDesc", "Data_Value", "Total"]]

In [10]:
obes_2011 = overall_data.loc[(overall_data["YearStart"] == 2011) & (overall_data["LocationDesc"] != "National") 
                         & (overall_data["LocationDesc"] != "District of Columbia") ]

obes_2011 = obes_2011.dropna()
obes_2011.count()

YearStart       50
LocationAbbr    50
LocationDesc    50
Data_Value      50
Total           50
dtype: int64

In [12]:
obes_2011.head()

,YearStart,LocationAbbr,LocationDesc,Data_Value,Total
3263,2011,AL,Alabama,32.0,Total
4418,2011,MD,Maryland,28.3,Total
7246,2011,TN,Tennessee,29.2,Total
7315,2011,MI,Michigan,31.3,Total
8777,2011,RI,Rhode Island,25.4,Total


In [19]:
state_geo = ("../Resources/us-states.json")
m = folium.Map(location=[37, -102], zoom_start=5)

# Add the color for the chloropleth:
m.choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=obes_2011,
    columns=['LocationAbbr', 'Data_Value'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Obesity Rate (%)'
    )

folium.LayerControl().add_to(m)
# m.save('#292_folium_chloropleth_USA1.html')